<a href="https://colab.research.google.com/github/Teacher-Heiko/course22/blob/master/ULMfit_DeBerta_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.text.all import *

In [2]:
%pip install -Uq datasets
from datasets import Dataset,DatasetDict
%pip install protobuf

In [3]:
%pip install -Uq accelerate

In [4]:
warnings.simplefilter('ignore')
# logging.disable(logging.WARNING)

In [5]:
# path = untar_data(URLs.IMDB)

In [6]:
# path

In [7]:
# path.ls()

In [8]:
# dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

In [9]:
# dls.show_batch()

In [10]:
model_nm = 'microsoft/deberta-v3-base'

In [11]:
 %pip install --no-cache-dir transformers sentencepiece

In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
tokz = AutoTokenizer.from_pretrained(model_nm, use_fast=False)
# Load the IMDb dataset
imdb_dataset = load_dataset('imdb')

# Tokenize the text data
tokenized_data = imdb_dataset.map(lambda example: tokz(example['text'], truncation=True, padding='max_length'), batched=True)

# Print the tokenized output
print(tokenized_data)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [29]:
tokenized_data['train'][0]['input_ids'][:20]

[1,
 273,
 11388,
 273,
 3846,
 51696,
 65516,
 271,
 3021,
 79919,
 1975,
 292,
 312,
 750,
 1106,
 401,
 265,
 305,
 262,
 9046]

In [30]:
tokz.decode(tokenized_data['train'][0]['input_ids'])[:20]

'[CLS] I rented I AM '

In [15]:
bs = 32
epochs = 4
lr = 8e-5 # 5e-6

In the following cell, you can set all the paramters for the training, depending on which model you use (see above).

* `bs`: The batch size for initial training - As it turned out a smaller batch size trains better (also see my blog post)
* `gradient_accumulation_steps`: Number of gradient accumulation steps for initial training
* `epochs`: Number of epochs for  initial training
* `reshuffle`: If you set this parameter to a value greater than zero, the data will be re-shuffled after the initial training for more training.
* `bs2`: The batch size for training after reshuffling
* `gradient_accumulation_steps2`: Number of gradient accumulation steps for training after reshuffling
* `epochs2`:  Number of epochs for training after reshuffling
* `train_all`: After initial training, train again on the whole dataset with half the learning rate

In [16]:
lr = 5e-6

if model_nm == 'microsoft/deberta-v3-small':
    bs = 4 #https://twitter.com/ylecun/status/989610208497360896
    gradient_accumulation_steps = 1
    epochs = 2

    reshuffle = 0
    bs2 = 4
    gradient_accumulation_steps2 = 1
    epochs2 = 1

if model_nm == 'microsoft/deberta-v3-base':
    bs = 4 #https://twitter.com/ylecun/status/989610208497360896
    gradient_accumulation_steps = 1
    epochs = 2

    reshuffle = 0
    bs2 = 4
    gradient_accumulation_steps2 = 1
    epochs2 = 1


if model_nm == 'microsoft/deberta-v3-large':
    bs = 4 #for large model
    gradient_accumulation_steps = 1
    epochs = 2

    reshuffle = 0
    bs2 = 4
    gradient_accumulation_steps2 = 1
    epochs2 = 1




In [17]:
from transformers import TrainingArguments,Trainer

# set TrainingArguments from the Transfromers library
args = TrainingArguments(output_dir='outputs', # directory where the model checkpoints and training-related files will be saved
                         learning_rate=lr, # initial learning rate for the optimizer.
                         warmup_ratio=0.1, # Ratio of total training steps used for a linear warmup from 0 to learning_rate
                         lr_scheduler_type='cosine', # type of learning rate scheduler to use
                         fp16=True, # Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training
                         evaluation_strategy='epoch', # strategy to use; Evaluation is done at the end of each epoch
                         per_device_train_batch_size=bs, # batch size per device during training
                         per_device_eval_batch_size=bs*2, # batch size for evaluation
                         num_train_epochs=epochs, # total number of training epochs to perform
                         weight_decay=0.01, # strength of weight decay
                         report_to='none' # The list of integrations to report the results and logs to.
                         )


In case you want to reshuffle and keep training

In [18]:
train_all = False

In [19]:
    # if train_all:
    #     model = trainer.model
    #     dds = tok_ds.train_test_split(0.25, seed=seed*(reshuffle_round+1))
    #     epochs=epochs2

    #     args = TrainingArguments('outputs', learning_rate=lr/2, warmup_ratio=0.1,
    #                              lr_scheduler_type='cosine', fp16=True,
    #                              evaluation_strategy="epoch", per_device_train_batch_size=bs2,
    #                              per_device_eval_batch_size=bs2*2, num_train_epochs=epochs2,
    #                              weight_decay=0.01, report_to='none',
    #                              gradient_accumulation_steps=gradient_accumulation_steps2,
    #                              save_steps=-1)

    #     trainer = Trainer(model, args, train_dataset=dataset,
    #                       eval_dataset=dds['test'],
    #                   tokenizer=tokz, compute_metrics=compute_metrics)

    #     trainer.train()

In [20]:
train_dataset = tokenized_data['train']
eval_dataset = tokenized_data['test']

In [21]:
from scipy.stats import pearsonr
def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    predictions = logits.argmax(axis=-1)
    pearson_corr, _ = pearsonr(predictions, labels)
    return {"pearson": pearson_corr}

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
model

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

will freeze the embeddings layer and the first 5 transformer layers.

In [24]:
# modules = [model.embeddings, *model.encoder.layer[:5]] #Replace 5 by what you want
# for module in modules:
#     for param in module.parameters():
#         param.requires_grad = False

In [25]:
trainer = Trainer(model, args, train_dataset=train_dataset, eval_dataset=eval_dataset,
                  tokenizer=tokz, compute_metrics=compute_metrics)

In [26]:
import gc

gc.collect() # garbage collect to free up memory
torch.cuda.empty_cache() # empty cache to free up memory
trainer.train() # train the model

Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored

In [ ]:
trainer.train()